In [ ]:

import sys
import os
from pathlib import Path

# For Jupyter or interactive use — use current working directory as script base
notebook_path = Path().resolve()

# Assume notebook is in a subfolder of the repo — go up one level
file_dir = notebook_path.parent.parent

# Set working directory to the repo root
os.chdir(file_dir)
print("Working directory set to:", Path.cwd())

sys.path.append(str(file_dir))


from netneurotools import datasets, stats
import pandas as pd
import numpy as np
import nibabel as nib
from statsmodels.stats.multitest import multipletests

data_dir = f"{file_dir}/data"

Working directory set to: /Users/melinatsotras/Desktop/submission


### Load data and atlas

In [ ]:
# Load label update file and filter for 400-parcel, 7-network resolution
label_updates_path = f'{file_dir}/Enrichment_Analysis/human/input/Update_20190916_component_name_changes.csv'
label_updates_df = pd.read_csv(label_updates_path)
label_updates_df = label_updates_df[label_updates_df['Resolution'] == '400Parcels_7Networks']

# Create a mapping from old to new parcel names
label_updates_dict = dict(zip(
    label_updates_df['Old parcel name'], 
    label_updates_df['New parcel name']
))

# Load Schaefer 400x7 network labels and rename columns for clarity
schaefer_path = f'{file_dir}/Enrichment_Analysis/human/input/schaefer400_7Networks_NaNDropped.csv'
schaefer_df = pd.read_csv(schaefer_path).rename(columns={'0': 'region', '1': 'network'})

# Set region as index and transpose for mapping
schaefer_df = schaefer_df.set_index('region').T

# Rename regions using updated parcel names
schaefer_df = schaefer_df.rename(columns=label_updates_dict).T

# Define mappings between network names and numbers
name_to_number = {
    'Visual': 1,
    'SomatoMotor': 2,
    'Dorsal Attention': 3,
    'Ventral Attention': 4,
    'Limbic': 5,
    'Frontoparietal': 6,
    'Default Mode': 7
}
number_to_name = {v: k for k, v in name_to_number.items()}

# Display the first few rows of the final dataframe
schaefer_df.head()


,network
region,
7Networks_LH_Vis_1,1
7Networks_LH_Vis_2,1
7Networks_LH_Vis_3,1
7Networks_LH_Vis_4,1
7Networks_LH_Vis_5,1


### Prepare cell type fractions parceled in Schaefer 400

In [ ]:
# Define paths to left and right hemisphere sphere surface files
lh_sphere_path = f'{data_dir}/gifti/fsaverage-human/tpl-fsaverage_den-41k_hemi-L_sphere.surf.gii'
rh_sphere_path = f'{data_dir}/gifti/fsaverage-human/tpl-fsaverage_den-41k_hemi-R_sphere.surf.gii'

# Load vertex-wise cell type labels
vertex_labels_path = f'{file_dir}/Enrichment_Analysis/human/input/Jorstad_vertex_wise_labels.csv'
vertex_wise_labels = pd.read_csv(vertex_labels_path)

# Extract the list of cell type names from the dataframe columns
cell_types = vertex_wise_labels.columns[2:]

# Load spherical coordinates for each hemisphere
lh_vertices = nib.load(lh_sphere_path).agg_data('pointset')
rh_vertices = nib.load(rh_sphere_path).agg_data('pointset')

# Combine left and right hemisphere vertices into a single array
vertices = np.concatenate([lh_vertices, rh_vertices])

# Create a hemisphere ID list: 0 for left hemisphere, 1 for right hemisphere
hemiid = [0] * lh_vertices.shape[0] + [1] * rh_vertices.shape[0]

# Output the list of cell types (optional, for inspection)
cell_types


Index(['ASTRO', 'CHANDELIER', 'ENDO', 'L2_3_IT', 'L4_IT', 'L5_ET', 'L5_IT',
       'L5_6_NP', 'L6_CT', 'L6_IT', 'L6_IT_CAR3', 'L6B', 'LAMP5', 'LAMP5_LHX6',
       'MICRO_PVM', 'OPC', 'OLIGO', 'PAX6', 'PVALB', 'SNCG', 'SST',
       'SST_CHODL', 'VLMC', 'VIP'],
      dtype='object')

In [ ]:
# Loop through each cell type in the Jorstad dataset
for i, cell_type in enumerate(cell_types):
    print(f"Processing: {cell_type}")

    # Generate 5000 spatially rotated (spin) permutations of vertex indices
    spins_vertices = stats.gen_spinsamples(
        vertices, hemiid, n_rotate=5000, seed=i
    )

    # Get the original brain map for this cell type as a NumPy array
    original_map = vertex_wise_labels[cell_type].to_numpy()

    # Create a DataFrame of the null distribution by applying spin permutations
    null_distribution = pd.DataFrame(original_map[spins_vertices])

    # Add the original Schaefer label column to the null distribution
    null_distribution['schaefer_label'] = vertex_wise_labels['label']

    # Save the null distribution to a CSV file
    output_path = (
        f"{file_dir}/output/Null_5000_Jorstad_{cell_type}_Schaefer400_NaNDropped.csv"
    )
    null_distribution.to_csv(output_path, index=False)


### Calculate Enrichment 

In [ ]:
# Load empirical parcellated cell-type enrichment data and map Yeo network labels
empirical_path = f'{file_dir}/Enrichment_Analysis/human/input/Empirical_Jorstad_Schaefer400_NaNDropped.csv'
empirical_jorstad = pd.read_csv(empirical_path).set_index('region')
empirical_jorstad['yeo_num'] = schaefer_df.network

# Compute the mean enrichment per Yeo network (empirical values)
true_value_jor = empirical_jorstad.groupby('yeo_num').mean()
true_value_jor.index = [number_to_name[num] for num in true_value_jor.index]

# Initialize DataFrames to hold means, standard deviations, z-scores, and p-values
means_jor = pd.DataFrame(0, index=name_to_number.keys(), columns=cell_types)
sds_jor = pd.DataFrame(0, index=name_to_number.keys(), columns=cell_types)
zscores_jor = pd.DataFrame(0, index=name_to_number.keys(), columns=cell_types)
pvals_jor = pd.DataFrame(0, index=name_to_number.keys(), columns=cell_types)
zscores_temp= pd.DataFrame(0, index=name_to_number.keys(), columns=cell_types)
pvals_temp = pd.DataFrame(0, index=name_to_number.keys(), columns=cell_types)

# Loop through each cell type and compute statistics
for cell_type in cell_types:
    print(f"Processing: {cell_type}")

    # Load the null distribution for the current cell type
    null_path = f"{file_dir}/output/Null_5000_Jorstad_{cell_type}_Schaefer400_NaNDropped.csv"
    null = pd.read_csv(null_path).groupby('yeo_label').mean() # groups each null map into 7-Network parcellation and takes average

    # Calculate means, standard deviations, z-scores, and p-values
    means_jor[cell_type] = null.mean(axis = 1, skipna = True)
    sds_jor[cell_type] = null.std(axis = 1, skipna = True)
    zscored_null = null.apply(lambda row: (row-means_jor[cell_type][row.name])/sds_jor[cell_type][row.name], axis=1)
    zscores_jor[cell_type] = (true_value_jor[cell_type]-means_jor[cell_type])/sds_jor[cell_type]

    # calculate proportion of absolute valued null values (zscored) that are greater than the absolute value of the true zscored value
    pvals_jor[cell_type] = zscored_null.abs().apply(lambda row: (row > zscores_jor.abs()[cell_type][row.name]).sum(), axis=1)/null.shape[1]
    

pvalues_jor = pvals_jor.copy()
zscores_jor = zscores_jor.T
pvalues_jor = pvalues_jor.T


Processing: ASTRO
Processing: CHANDELIER
Processing: ENDO
Processing: L2_3_IT
Processing: L4_IT
Processing: L5_ET
Processing: L5_IT
Processing: L5_6_NP
Processing: L6_CT
Processing: L6_IT
Processing: L6_IT_CAR3
Processing: L6B
Processing: LAMP5
Processing: LAMP5_LHX6
Processing: MICRO_PVM
Processing: OPC
Processing: OLIGO
Processing: PAX6
Processing: PVALB
Processing: SNCG
Processing: SST
Processing: SST_CHODL
Processing: VLMC
Processing: VIP


In [13]:
# FDR correction
fdr_p_value = pvalues_jor.copy()

for col in pvalues_jor.columns:
    _, fdr_corrected_p, _, _ = multipletests(pvalues_jor[col], method='fdr_bh')
    fdr_p_value[col] = fdr_corrected_p

display(fdr_p_value)
fdr_p_value.to_csv(f"{file_dir}/Enrichment_Analysis/human/enrichment_analysis_fdr_pvalues_jorstad_5000maps.csv")


,Visual,SomatoMotor,Dorsal Attention,Ventral Attention,Limbic,Frontoparietal,Default Mode
ASTRO,0.475938,0.445662,0.978400,0.580800,0.206400,0.869800,0.534720
CHANDELIER,0.475938,0.477600,0.952070,0.168600,0.089400,0.869800,0.976400
ENDO,0.676080,0.376582,0.952070,0.856200,0.064800,0.869800,0.706560
L2_3_IT,0.925200,0.176000,0.249600,0.671314,0.734836,0.249600,0.976400
L4_IT,0.475938,0.557929,0.952070,0.152000,0.269169,0.612800,0.504800
L5_ET,0.708800,0.267600,0.952070,0.856200,0.393900,0.869800,0.976400
L5_IT,0.475938,0.176000,0.361600,0.395576,0.639840,0.033600,0.400000
L5_6_NP,0.676080,0.177600,0.952070,0.273257,0.921400,0.869800,0.369600
L6_CT,0.676080,0.224800,0.896000,0.281700,0.734836,0.612800,0.976400
L6_IT,0.676080,0.557929,0.952070,0.147200,0.639840,0.249600,0.602182


In [14]:
display(zscores_jor)
zscores_jor.to_csv(f"{file_dir}/Enrichment_Analysis/human/enrichment_analysis_zscores_jorstad_5000maps.csv")

,Visual,SomatoMotor,Dorsal Attention,Ventral Attention,Limbic,Frontoparietal,Default Mode
ASTRO,-1.386105,-1.205037,0.026888,0.745203,1.553155,-0.475081,1.307510
CHANDELIER,-1.017570,-1.038418,-0.792764,1.817734,2.478581,-0.554973,-0.117345
ENDO,0.818463,1.408802,0.206740,0.215712,-2.396949,-0.227875,-0.767833
L2_3_IT,0.098026,-2.170779,2.499428,-0.554673,-0.419932,2.161122,0.281452
L4_IT,2.308543,0.905493,-0.315745,-1.981883,-1.236086,-1.366831,-1.554530
L5_ET,-0.612551,1.568445,0.221492,0.210200,-0.980304,0.173619,-0.358274
L5_IT,-1.645084,-2.302217,1.970882,-1.064783,0.648819,2.981739,2.051551
L5_6_NP,-0.633768,-2.025400,-0.203675,1.421489,-0.105435,0.547171,2.333021
L6_CT,-0.610634,1.791592,-1.045323,1.366735,-0.484856,-1.434968,-0.163726
L6_IT,-0.776175,0.951853,-0.339047,2.340523,0.648589,-2.194593,-1.111085
